In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Zbiór danych pochodzi ze strony https://nijianmo.github.io/amazon/index.html.

Wykorzystano z niej również instrukcję
odnośnie sposobu w jaki zbiór załadować w celu dalszych operacji na nim.

In [ ]:
import pandas as pd
import gzip
import json
import re

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/drive/MyDrive/dataset.json.gz')

In [ ]:
dataset = df.filter(['reviewText', 'overall'])
dataset = dataset.rename(columns={'reviewText': 'text', 'overall': 'score'})

In [ ]:
dataset = dataset[dataset['score'] != 3]
dataset['score'] = dataset['score'].apply(lambda x: 1 if x > 3 else 0)

In [ ]:
patterns = [
  "\w*\d\w*", #removing digits
  r"(http\S+)", #removing URLs
  r"([\w\.\-\_]+@[\w\.\-\_]+)", #removing e-mails
  r"([^a-zA-Z\s])" #removing special characters
  r"(@[A-Za-z0-9_]+)", #removing @pings
  r"(#[A-Za-z0-9_]+)" #removing hashtags
  r"[\n\t\s]*"] #Remove all white \t spaces, new lines \n and tabs \t

for pattern in patterns:
  dataset['text'] = dataset['text'].apply(lambda x: re.sub(pattern, '', str(x)))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

dataset['text']=dataset['text'].apply(lambda x: remove_emoji(x))

In [ ]:
dataset = dataset.dropna()
dataset = dataset.drop_duplicates(subset=['text'], keep='first')
dataset = dataset[dataset['text'].str.len() > 0]
dataset['text'] = dataset['text'].str.strip() #remove tabs (trailing and leading spaces)

In [ ]:
dataset.to_csv(open('/content/drive/MyDrive/dataset.csv', 'wb'), index=False)